## Evaluating OCR quality

In this notebook we evaluate OCR quality on a sample image part for evaluation(source: [Deutsche Zeitung, Ausgaben am Montag, 23.12.1918](https://zefys.staatsbibliothek-berlin.de/kalender/auswahl/date/1918-12-23/30744015/)):


![sample.jpg](sample.jpg)

### Imports and OCR

Let us OCR it first:

In [ ]:
import pytesseract
from PIL import Image

In [ ]:
ocr_output = pytesseract.image_to_string(Image.open('sample.jpg'), lang='frk')  # using German fraktur OCR model

In [ ]:
print(ocr_output)

### Manually create  the 'ground truth' to evaluate against

In [ ]:
ground_truth = input('Please insert corrected string: ')

In [ ]:
print(ground_truth)

### Calculate precision, recall, and F1-measure

In [ ]:
from aux.measure_ocr_quality import measure_ocr_quality

In [ ]:
precision, recall, f_score = measure_ocr_quality(ocr_output, ground_truth)

In [ ]:
print(f'Precision: {round(precision, 4)}\nRecall: {round(recall, 4)}\nF1-score: {round(f_score, 4)}')